In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LayerNormalization, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed
import tensorflow as tf
from tensorflow import keras

In [2]:
import keras_tuner as kt

In [3]:
Z_eff = pd.read_csv('csv/data_Z_eff.csv',header=None)
R_eff = pd.read_csv('csv/data_R_eff.csv',header = None)
R_org_eff = pd.read_csv('csv/data_R_org_eff.csv',header = None)
group_ind = pd.read_csv('csv/data_group_ind.csv',header =None)

Z_eff2 = Z_eff.copy()
R_eff2 = R_eff.copy()
R_org_eff2 = R_org_eff.copy()
group_ind2 = group_ind.copy()

In [4]:
Z_eff.shape

(2219316, 64)

In [5]:
t_train_start = list(range(0,41*12+1,12))
t_train_end =[x+120 for x in t_train_start]
t_val_start= [x for x in t_train_end]
t_val_end = [x+60 for x in t_val_start]
t_test_start = [x for x in t_val_end]
t_test_end = [x+12 for x in t_test_start]
t_test_end[41]=678

In [6]:
train_start = group_ind.iloc[t_train_start][0].to_list()
train_end = group_ind.iloc[t_train_end][0].to_list()
test_start = group_ind.iloc[t_test_start][0].to_list()
test_end = group_ind.iloc[t_test_end][0].to_list()
val_start = group_ind.iloc[t_val_start][0].to_list()
val_end = group_ind.iloc[t_val_end][0].to_list()                         

In [7]:
def portf_ret(preds, y_org):
    list_buy = [idx for idx, val in preds if val > np.percentile(preds,90)]
    list_buy = [idx for idx, val in preds if val < np.percentile(preds,10)]
    portf_ret_t = np.mean(y_true_t[list_buy]) - np.mean(y_true_t[list_sell])
    return portf_ret_t

In [8]:
def model_builder(hp):
    model = Sequential()
    model.add(LayerNormalization())
    
    for i in range(hp.Int('layers', 3, 3)):
        #explores possible combinations of 3 layers with units ranging between 64 and 128 with a step change of 16
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), 64, 128, step=16),
                                    activation=hp.Choice('act_' + str(i), ['relu'])))
        
    model.add(BatchNormalization())
    model.add(keras.layers.Dense(1))
    hp_learning_rate = hp.Choice('learning_rate', values=[0.005, 0.001, 0.0005])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='mse')
    
    return model

In [9]:
y_pred = []
y_pred_val = []
y_pred_test = []
loss = []
model_list = []
y_org = []
y_org_val = []
y_org_test = []
y_reff = []
y_reff_val = []
y_reff_test = []
portf_ret_nn_test = []
portf_ret_nn_test2 = []
portf_ret_nn_testFinal = []
R_pred=R_org_eff
R_pred2 = R_org_eff

#to set global and operation seeds
seed(1)
tf.compat.v1.random.set_random_seed(1234)
tf.random.set_seed(5)
tf.random.uniform([1])
tf.random.normal([1])

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.22652863], dtype=float32)>

In [11]:
counter = 0
for retrain_idx in range(42):
    X = Z_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Z_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    X_test = Z_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    X_val = X_val.loc[:,X_idx]
    X_test = X_test.loc[:,X_idx]
    y =R_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    y_val = R_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    y_test = R_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    yorg = R_org_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    yorg_val = R_org_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    yorg_test=R_org_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    
    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
    
    #Keras Tuner
    #********************************************************************************************************** 
    
    tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=12,
                     factor=3,
                     directory="models",
                         seed = 1,
                     project_name=('final_'+str(counter)))
    
    
    
    tuner.search(x=X,y=y,validation_data=(X_val,y_val), epochs=30, validation_split=0.20, callbacks=[early_stop])
    
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
    
    #_________________________________________________________________________________
    #The following lines are to find the best parameters and best epoch the first time. We ignored best epoch and relied on our low early stop patience
    #_________________________________________________________________________________
    #modelTune = tuner.hypermodel.build(best_hps)
    #history = modelTune.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=64, epochs=20, validation_split=0.2,callbacks=[early_stop])

    #val_acc_per_epoch = history.history['val_loss']
    #best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
    #print('Best epoch: %d' % (best_epoch,))
    
    #_________________________________________________________________________________
    #********************************************************************************************************** 
    
    model = tuner.hypermodel.build(best_hps)

    # Retrain the model
    history = model.fit(x=X,y=y,validation_data=(X_val,y_val), epochs=20,  batch_size=128, validation_split=0.2,callbacks=[early_stop])
   
    preds = model.predict(X_test).flatten()
    y_org_test = yorg_test[0].to_list()
    portf_ret_nn_test = []
    
    for t in range(t_test_start[retrain_idx], t_test_end[retrain_idx]):
        month_start = group_ind.iloc[t][0] - group_ind.iloc[t_test_start[retrain_idx]][0]
        month_end = group_ind.iloc[t+1][0] - group_ind.iloc[t_test_start[retrain_idx]][0]
        preds_month = preds[month_start:month_end]
        yorg_month = y_org_test[month_start:month_end]
        list_buy = [i for i, val in enumerate(preds_month) if val > np.quantile(preds_month,.9)]
        list_sell = [i for i, val in enumerate(preds_month) if val < np.quantile(preds_month,.1)]
        print(t,month_start,month_end,len(preds_month),len(list_buy),len(list_sell))
        portf_ret_nn_test.append(np.mean([yorg_month[i] for i in list_buy]) - np.mean([yorg_month[i] for i in list_sell]))
        #print(counter,np.nanmean(portf_ret_nn_test)/np.nanstd(portf_ret_nn_test)*np.sqrt(12))
    loss.append(min(history.history['val_loss']))
    
    
    #Running the second base manually tuned model to handle bad sharpe due to some bad models
    #************************************************************************************************************
    X2 = Z_eff2.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    X_idx2 = X2.apply(pd.Series.nunique) != 1
    X2 = X2.loc[:,X_idx2]
    X_val2 = Z_eff2.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    X_test2 = Z_eff2.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    X_val2 = X_val.loc[:,X_idx2]
    X_test2 = X_test2.loc[:,X_idx2]
    y2 =R_eff2.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    y_val2 = R_eff2.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    y_test2 = R_eff2.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    yorg2 = R_org_eff2.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    yorg_val2 = R_org_eff2.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    yorg_test2 =R_org_eff2.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    
    
    model2 = Sequential()
    model2.add(LayerNormalization())
    model2.add(Dense(64,activation='relu'))
    model2.add(BatchNormalization())
    model2.add(Dense(32,activation='relu'))
    model2.add(BatchNormalization())
    model2.add(Dense(16,activation='relu'))
    model2.add(BatchNormalization())
    model2.add(Dense(1))
    model2.compile(optimizer='adam',loss='mse')
    early_stop2 = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
    history2 = model2.fit(x=X2,y=y2,validation_data=(X_val2,y_val2), batch_size=64, epochs=50,verbose=1,callbacks=[early_stop2])
    preds2 = model2.predict(X_test2).flatten()
    y_org_test2 = yorg_test2[0].to_list()
    R_pred2.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]=pd.DataFrame(preds2)
    
    portf_ret_nn_test2 = []
    for t in range(t_test_start[retrain_idx], t_test_end[retrain_idx]):
        month_start2 = group_ind.iloc[t][0] - group_ind.iloc[t_test_start[retrain_idx]][0]
        month_end2 = group_ind.iloc[t+1][0] - group_ind.iloc[t_test_start[retrain_idx]][0]
        preds_month2 = preds2[month_start2:month_end2]
        yorg_month2 = y_org_test2[month_start2:month_end2]
        list_buy2 = [i for i, val in enumerate(preds_month2) if val > np.quantile(preds_month2,.9)]
        list_sell2 = [i for i, val in enumerate(preds_month2) if val < np.quantile(preds_month2,.1)]
        print(t,month_start2,month_end2,len(preds_month2),len(list_buy2),len(list_sell2))
        portf_ret_nn_test2.append(np.mean([yorg_month2[i] for i in list_buy2]) - np.mean([yorg_month2[i] for i in list_sell2]))          
        #print("--",counter,np.nanmean(portf_ret_nn_test2)/np.nanstd(portf_ret_nn_test2)*np.sqrt(12))
    loss.append(min(history.history['val_loss']))
    
    
    #Comparing the sharpe's from 2 models to pick best for each year
    #***********************************************************************************************************
    if((np.nanmean(portf_ret_nn_test)/np.nanstd(portf_ret_nn_test)*np.sqrt(12)) > (np.nanmean(portf_ret_nn_test2)/np.nanstd(portf_ret_nn_test2)*np.sqrt(12))):
        for i in range(len(portf_ret_nn_test)):
            R_pred.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]=pd.DataFrame(preds)
            portf_ret_nn_testFinal.append(portf_ret_nn_test[i])
            model_list.append(model)
    else:
        for i in range(len(portf_ret_nn_test2)):
            R_pred.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]=pd.DataFrame(preds2)
            portf_ret_nn_testFinal.append(portf_ret_nn_test2[i])
            model_list.append(model2)
    
    print("final",counter,(np.nanmean(portf_ret_nn_testFinal)/np.nanstd(portf_ret_nn_testFinal)*np.sqrt(12)))
    
    counter = counter + 1

Trial 31 Complete [00h 01m 10s]
val_loss: 0.010269525460898876

Best val_loss So Far: 0.010234387591481209
Total elapsed time: 00h 09m 08s
INFO:tensorflow:Oracle triggered exit
hi 112

    The hyperparameter search is complete. The optimal number of units in the first densely-connected
    layer is 112 and the optimal learning rate for the optimizer
    is 0.005.
    
Epoch 1/20
2687/2687 [==============================] - 6s 2ms/step - loss: 0.0143 - val_loss: 0.0110
Epoch 2/20
2687/2687 [==============================] - 5s 2ms/step - loss: 0.0095 - val_loss: 0.0105
Epoch 3/20
2687/2687 [==============================] - 5s 2ms/step - loss: 0.0091 - val_loss: 0.0105
Epoch 4/20
2687/2687 [==============================] - 5s 2ms/step - loss: 0.0090 - val_loss: 0.0103
Epoch 5/20
2687/2687 [==============================] - 5s 2ms/step - loss: 0.0089 - val_loss: 0.0103
Epoch 6/20
2687/2687 [==============================] - 5s 2ms/step - loss: 0.0089 - val_loss: 0.0102
Epoch 7/20
2687/2

<ipython-input-11-f7fb299b1933>:65: RuntimeWarning: divide by zero encountered in double_scalars
  print(counter,np.nanmean(portf_ret_nn_test)/np.nanstd(portf_ret_nn_test)*np.sqrt(12))


19 inf
19 55.21205176845808
19 26.557630037928607
19 14.725654816727976
19 15.532353991507557
19 16.66011271459948
19 13.786007475741329
19 12.560830484289582
19 13.328139210369857
19 13.885166808344975
19 14.032693477835567
19 13.248715043716736
Epoch 1/50
6716/6716 [==============================] - 12s 2ms/step - loss: 0.0190 - val_loss: 0.0095
Epoch 2/50
6716/6716 [==============================] - 12s 2ms/step - loss: 0.0094 - val_loss: 0.0094
Epoch 3/50
6716/6716 [==============================] - 12s 2ms/step - loss: 0.0093 - val_loss: 0.0092
Epoch 4/50
6716/6716 [==============================] - 12s 2ms/step - loss: 0.0092 - val_loss: 0.0093
Epoch 5/50
6716/6716 [==============================] - 12s 2ms/step - loss: 0.0092 - val_loss: 0.0092
Epoch 6/50
6716/6716 [==============================] - 12s 2ms/step - loss: 0.0092 - val_loss: 0.0093
Epoch 00006: early stopping


<ipython-input-11-f7fb299b1933>:109: RuntimeWarning: divide by zero encountered in double_scalars
  print("--",counter,np.nanmean(portf_ret_nn_test2)/np.nanstd(portf_ret_nn_test2)*np.sqrt(12))


-- 19 inf
-- 19 79.72075347077887
-- 19 54.45523318533382
-- 19 50.6636007242917
-- 19 25.514478961779982
-- 19 27.919396850331207
-- 19 22.306925951567127
-- 19 14.137131881859696
-- 19 14.623059086357104
-- 19 15.039445668728286
-- 19 15.635183362353567
-- 19 14.825749755273087
final 19 7.62187771059359
INFO:tensorflow:Reloading Oracle from existing project models\final_20\oracle.json
INFO:tensorflow:Reloading Tuner from models\final_20\tuner0.json
INFO:tensorflow:Oracle triggered exit
hi 96

    The hyperparameter search is complete. The optimal number of units in the first densely-connected
    layer is 96 and the optimal learning rate for the optimizer
    is 0.0005.
    
Epoch 1/20
2734/2734 [==============================] - 5s 2ms/step - loss: 0.0252 - val_loss: 0.0124
Epoch 2/20
2734/2734 [==============================] - 5s 2ms/step - loss: 0.0095 - val_loss: 0.0096
Epoch 3/20
2734/2734 [==============================] - 5s 2ms/step - loss: 0.0092 - val_loss: 0.0111
Epoch 4/

...

-- 40 inf
-- 40 7.410617631213217
-- 40 6.28157349381245
-- 40 7.200363787252667
-- 40 7.321826834341759
-- 40 7.005730039106123
-- 40 3.58437440339797
-- 40 3.9427331021185013
-- 40 4.299640374215136
-- 40 4.56431166795256
-- 40 4.614076401959099
-- 40 4.796149264825338
final 40 5.506826852393554
INFO:tensorflow:Reloading Oracle from existing project models\final_41\oracle.json
INFO:tensorflow:Reloading Tuner from models\final_41\tuner0.json
INFO:tensorflow:Oracle triggered exit
hi 80

    The hyperparameter search is complete. The optimal number of units in the first densely-connected
    layer is 80 and the optimal learning rate for the optimizer
    is 0.005.
    
Epoch 1/20
2416/2416 [==============================] - 5s 2ms/step - loss: 0.0150 - val_loss: 0.0074
Epoch 2/20
2416/2416 [==============================] - 4s 2ms/step - loss: 0.0099 - val_loss: 0.0077
Epoch 3/20
2416/2416 [==============================] - 4s 2ms/step - loss: 0.0097 - val_loss: 0.0073
Epoch 4/20
2416/2

<ipython-input-11-f7fb299b1933>:65: RuntimeWarning: divide by zero encountered in double_scalars
  print(counter,np.nanmean(portf_ret_nn_test)/np.nanstd(portf_ret_nn_test)*np.sqrt(12))


41 inf
41 27.86437654396359
41 9.095974738289204
41 7.490063592951709
41 7.194919118064288
41 7.727120483811948
Epoch 1/50
6039/6039 [==============================] - 10s 2ms/step - loss: 0.0243 - val_loss: 0.0087
Epoch 2/50
6039/6039 [==============================] - 9s 2ms/step - loss: 0.0090 - val_loss: 0.0086
Epoch 3/50
6039/6039 [==============================] - 9s 2ms/step - loss: 0.0089 - val_loss: 0.0086
Epoch 4/50
6039/6039 [==============================] - 10s 2ms/step - loss: 0.0089 - val_loss: 0.0087
Epoch 5/50
6039/6039 [==============================] - 10s 2ms/step - loss: 0.0088 - val_loss: 0.0088
Epoch 00005: early stopping


<ipython-input-11-f7fb299b1933>:109: RuntimeWarning: divide by zero encountered in double_scalars
  print("--",counter,np.nanmean(portf_ret_nn_test2)/np.nanstd(portf_ret_nn_test2)*np.sqrt(12))


-- 41 inf
-- 41 4.635940507987061
-- 41 4.810191808950715
-- 41 4.123841781435733
-- 41 4.910595553392723
-- 41 5.362717535992839
final 41 5.501731394083942


In [12]:
np.nanmean(portf_ret_nn_testFinal)/np.nanstd(portf_ret_nn_testFinal)*np.sqrt(12)

5.501731394083942